# Optimisation problem
Consider a problem of finding a minimum of a function. The generic solution is probing the function and gradually looking for smaller values.

Next, consider a problem: if y=f(x), then at which x the y is equal to z. Finding the z is called the optimization problem. 

One of the tools for tacking this problem is Particle Swarm Optimisation (PSO). It is a population-based stochastic optimisation technique inspired by the social behaviour of birds.
Here we will see how to set up a solution using a simple minimiser and a more complex PSO algorithm.

The example presented here is a 2D problem. We will be looking for a pair of the process parameters of the deposition process (tau_r, p_o) that will result in a deposit with the given pair of features (r_max, R_ind). The function that we will be minimising is the difference between the target values and the values calculated by the model.

In [ ]:
import numpy as np
from scipy.optimize import minimize
from backend.processclass2 import Experiment1D_Dimensionless
from backend.adaptive_tools import learner_interpolator, learner_load_full
from backend.pso_optimizer_class_base import PSOptimizerMP

## 1. Define the problem
Firstly, we define the experiment by creating an instance of the Experiment1D_Dimensionless class. We set the parameters of the experiment that are important for the current case.

In [ ]:
pr_d = Experiment1D_Dimensionless()
pr_d.step = 1
pr_d.tau_r = 5000
pr_d.p_o = 2
pr_d.fwhm = 200
pr_d.f0 = 1e7
pr_d.beam_type = 'super_gauss'
pr_d.order = 4

Secondly, we define the target values of the features that we want to achieve. In this case, we are looking for the process parameters that will result in a deposit with the given r_max and R_ind values.

In [16]:
r_max = [0.829] #  Normalaized peak position
R_ind = [0.333] #  Relative indent

Using the normalized peak position $r_{max}$ and the relative indent $R_{ind}$, we determine the values of depletion $\tau$ and diffusive replenishment $\rho_o$.
This can be achieved in two ways:
* using pre-calculated maps of $r_{max}$ and $R_{ind}$ resolved on a $\tau$ – $\rho_o$ grid.
* using a *Reaction-Diffusion Equation* (RDE) solver and a minimization function.

Using maps (interpolation functions) is substantially faster, but it takes a substantial amount of time to establish them. 
Moreover, one set of two maps is valid only for a certain order of the Super Gaussian (SG) intensity profile.

Because solving a RDE is an expensive operation, it is feasable to use it only with a minimization function or any other function that does not require a large number of evaluations.
But a simple minimization function suffers from a huge problem that all the fancy optimisation algorithms try to alleviate: entrapmnent in a local minimum.

In this regard, PSO is a better choice. Due to its stochastic nature, it is less likely to get stuck in a local minimum. But it requires substantually more evaluations of the function and thus is more computationally expensive.

Here both approaches are described. The user is encouraged to change the bounds and the hyperparameters to see their effect on the results.

## 2. Building a solution
### 2.1 Using the minimization function
#### 2.1.1 Define the function to minimize
Any optimization problem has to be brought down to a minimization problem. In this case, we are looking for the process parameters that will result in a deposit with the given r_max and R_ind values. The function that we will be minimizing is the difference between the target values and the values calculated by the model.

In [ ]:
def obj_func(xy):
    pr_d.p_o, pr_d.tau_r = xy
    pr_d.solve_steady_state()
    r_m = pr_d.r_max/pr_d.fwhm
    R_i = pr_d.R_ind
    print(f'r_m, R_i: {r_m:.3f},{R_i:.3f}, tau, p_o: {pr_d.tau_r:.3f},{pr_d.p_o:.3f}')
    return abs(r_m - z11) + abs(R_i - z22) # + abs(phi - zz3)

#### 2.1.2 Define the bounds
The bounds are the limits of the search space. It is highly recommended to set them as close to the expected values as possible. This will reduce the number of evaluations and the probability of getting stuck in a local minimum.

In [ ]:
bonds = [(0.1, 2), (1000, 10000)]

#### 2.1.3 Perform the minimization
Finally, we build up the optimisation script and run it. The script will print the current iteration and the values of the current step.

In [ ]:
def print_progress(xk):
    print(f"Iteration: {len(progress)}, {xk}")
    progress.append(xk)
# Perform the minimization to find the (x, y) coordinates
initial_guess = [1, 9000]  # Starting point for optimization
res = []
progress = []
for z11, z22 in zip(r_max, R_ind):
    result = minimize(obj_func, initial_guess, method='L-BFGS-B', bounds=bonds, callback=print_progress)
    res.append(result.x)
    print(f'r_max, R_ind: {z11:.3f},{z22:.3f}, x,y:{result.x.astype(np.float32):}')
res = np.array(res)
print(res)

### 2.2 Using the PSO algorithm 
#### 2.2.1 Open the maps (interpolation functions)
Because for the PSO algorithm we need to evaluate the function many times, it is better to use the maps. Here we open the maps.

In [ ]:
r_max_map_fname = r'../examples/r_max_interp_1.0.int'
R_ind_map_fname = r'../examples/R_ind_interp_1.0.int'
r_max_map_learner = learner_load_full(r_max_map_fname)
R_ind_map_learner = learner_load_full(R_ind_map_fname)
r_max_map = learner_interpolator(r_max_map_learner)
R_ind_map = learner_interpolator(R_ind_map_learner)

#### 2.2.2 Define the function to minimize
For the PSO algorithm, we have to define the function that will be minimized. The function has to return the value of the objective function.
The objective function for the PSO has to accept and return certain arguments as shown below. 

In [ ]:
def operator_func_map(ref_names, maps, param_names, param_vars):
    """
    Set parameters and run a simulation, process-safe
    :param ref_names:
    :param pr: experiment instance
    :param param_names: parameters names to set
    :param param_vars: parameters values to set
    :return:
    """    
    r_max_map = maps[0]
    R_ind_map = maps[1]
    r_m = r_max_map(param_vars[0], param_vars[1])
    R_i = R_ind_map(param_vars[0], param_vars[1])

    result = {}
    vals = [r_m, R_i] + param_vars.tolist()
    names = ref_names + param_names
    for name, val in zip(names, vals):
        result[name] = val

    return result

#### 2.2.3 Define the bounds

In [ ]:
bonds = [(0.1, 2), (1000, 10000)]

#### 2.2.4 Perform the PSO
There are several hyperparameters that can be set to improve the performance of the PSO algorithm. The most important are the number of particles and the number of iterations.
To that, there are two additional parameters that can be set: c1 and c2. They are the cognitive and social parameters, respectively. They control the influence of the personal best and the global best on the particle's velocity.

In [ ]:
res = []
for x, y in zip(r_max, R_ind):
    pso = PSOptimizerMP(2, 2, n_particles=50, n_cores=1, n_iterations=50) # more than one core is not supported in the notebooks
    pso.set_reference((x, 'r_max_n'), (y, 'R_ind'))
    pso.set_variables((0.01, 15, 0.1, 'p_o'), (10, 2000, 5, 'tau_r'))  # setting the bounds. The third parameter is related to 'velocity' or how much the particle is allowed to accelerate.
    pso.visualize = False
    # The cognitive and social parameters
    pso.c1 = 2
    pso.c2 = 1
    pso.set_operator(operator_func_map)
    pso.configure_operator(['r_max_n', 'R_ind'], (r_max_map, R_ind_map))
    result = pso.start_optimization()
    fit_score = pso.global_best_value
    res.append(result)
print(res)
